In [2]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import prepare_drafts as drafts


def _add_rank_actual(df_drafts: pd.DataFrame, df_ranks: pd.DataFrame) -> pd.DataFrame:
    """ 
    Adds the actual rank of each player for every draft.
    This is to determine how often actual rank differs from derived.
    """
    
    df_base = df_drafts.copy()
    df_ranks = df_ranks.copy()

    keep_vars = ['player', 'date', 'adp', 'rank_actual']
    df_ranks = df_ranks[keep_vars]

    rename_vars = {'player': 'drafted_player'
                    , 'adp': 'projection_adp'
                    , 'date': 'ranks_draft_date'}
    df_ranks.rename(columns=rename_vars, inplace=True)

    df = pd.merge(df_base, df_ranks, how='left'
                , on=['drafted_player', 'projection_adp', 'ranks_draft_date'])

    return df


def _add_rank_derived(df: pd.DataFrame, by_var: str=None) -> pd.DataFrame:
    """ 
    Adds derived rank for each draft/player based off adp.
    Note that even early round derived ranks won't align
    with actual due to multiple players having the same ADP.
    """

    if by_var is None:
        by_var = 'draft_id'

    df = df.copy()
    df.sort_values(by=[by_var, 'projection_adp', 'number'], inplace=True)

    df['rank_derived'] = 1
    df['rank_derived'] = df.groupby(by_var)['rank_derived'].cumsum()

    # df.sort_values(by=[by_var, 'number'], inplace=True)

    return df


def add_ranks(df_drafts: pd.DataFrame, df_ranks: pd.DataFrame) -> pd.DataFrame:
    """ Adds all rank versions to df_drafts. """
    
    df = _add_rank_actual(df_drafts, df_ranks)
    df = _add_rank_derived(df)

    return df


def create_lookup(df: pd.DataFrame, col: str, lookup_type: str) -> pd.DataFrame:
    """ Creates a section of the lookups data. """
    df = df.copy()
    
    keep_vars = ['draft_year', col]
    df = df_w_rank[keep_vars].drop_duplicates(subset=keep_vars)

    df.rename(columns={col: 'drafts_val'}, inplace=True)
    df['ranks_val'] = df['drafts_val']
    df['lookup_type'] = lookup_type

    df = df.loc[df['drafts_val'].notnull()]

    df = df[['lookup_type', 'draft_year', 'drafts_val', 'ranks_val']]

    return df


def export_lookups(df: pd.DataFrame, folder_path: str, file_name: str) -> None:
    df_player = create_lookup(df, 'full_name', 'player')
    df_team = create_lookup(df, 'team_name', 'team')
    df_pos = create_lookup(df, 'position', 'position')

    df = pd.concat([df_player, df_team, df_pos])

    df.to_csv(path.join(folder_path, file_name), index=False)


pd.set_option('display.max_rows', 300)

DRAFTS_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model/data'

df_drafts = drafts.read_drafts(DRAFTS_FOLDER, [2021])
df_updated_types = drafts.update_dtypes(df_drafts)
df_complete_players = drafts.drafts_w_player_data(df_updated_types)
df_draft_attrs = drafts.add_draft_attrs(df_complete_players)
df_w_rank_date = drafts._add_rank_draft_date(df_draft_attrs)
df_w_rank = _add_rank_derived(df_w_rank_date)

df_w_rank

,appearance_id,created_at,draft_entry_id,number,projection_adp,draft_id,position,team_name,draft_source,draft_title,full_name,num_teams,round,round_pick,draft_datetime,draft_date,draft_year,ranks_draft_date,rank_derived
14688,78a5634d-93aa-4cd9-b1af-dfb829df452c,2021-08-12 00:24:20,0a7b9aa4-dcd4-45fa-88c0-dcf6e11ab2ae,1,1.0,0072c9f8-d403-46e3-818c-ecfccc6dfa43,RB,Carolina Panthers,sit_and_go,Megan Draft 2,Christian McCaffrey,12,1,1,2021-08-12 00:24:20,2021-08-12,2021,2021-08-11,1
14689,a971451b-f89e-498f-a6b4-060854880ed6,2021-08-12 00:24:39,4fd3d06e-329a-42ef-a0ed-7d65a66096c9,2,2.1,0072c9f8-d403-46e3-818c-ecfccc6dfa43,RB,Minnesota Vikings,sit_and_go,Megan Draft 2,Dalvin Cook,12,1,2,2021-08-12 00:24:20,2021-08-12,2021,2021-08-11,2
14690,53565b50-b52d-4c4f-bcf7-ee96476f4bbc,2021-08-12 00:24:49,bf7b3739-8817-4da3-87a4-1575a3d46b8e,3,3.8,0072c9f8-d403-46e3-818c-ecfccc6dfa43,RB,Tennessee Titans,sit_and_go,Megan Draft 2,Derrick Henry,12,1,3,2021-08-12 00:24:20,2021-08-12,2021,2021-08-11,3
14691,30149020-3a70-47ce-a8e2-b52f0a5770fa,2021-08-12 00:25:03,873a2273-732e-4110-8af2-81b88a0e2ffa,4,4.3,0072c9f8-d403-46e3-818c-ecfccc6dfa43,RB,New Orleans Saints,sit_and_go,Megan Draft 2,Alvin Kamara,12,1,4,2021-08-12 00:24:20,2021-08-12,2021,2021-08-11,4
14692,c5897583-8409-4f22-a2af-e0eb91deaadd,2021-08-12 00:25:19,f02675ba-48d9-4370-836c-a9aefb666819,5,5.7,0072c9f8-d403-46e3-818c-ecfccc6dfa43,RB,Dallas Cowboys,sit_and_go,Megan Draft 2,Ezekiel Elliott,12,1,5,2021-08-12 00:24:20,2021-08-12,2021,2021-08-11,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15086,ca04265c-c408-4d13-869d-101f4931214a,2021-07-18 21:22:59,d538d698-3795-40a7-b730-5d0928288049,183,211.4,fba9d9fe-fa23-40c4-91c0-ee66e853ac07,QB,Carolina Panthers,sit_and_go,NaN,Cam Newton,12,16,3,2021-07-18 20:38:10,2021-07-18,2021,2021-07-18,212
15097,937435d2-8d97-43dc-b893-e61427d971a4,2021-07-18 21:25:34,4c02707f-5f32-4b0b-9e50-f3f4aa8fca71,194,212.3,fba9d9fe-fa23-40c4-91c0-ee66e853ac07,TE,Atlanta Falcons,sit_and_go,NaN,Hayden Hurst,12,17,2,2021-07-18 20:38:10,2021-07-18,2021,2021-07-18,213
15087,06d3e22d-8e89-4e41-a10f-52b649e07f75,2021-07-18 21:23:30,bbebd25d-d9db-4c31-b4c3-e763ad71ce25,184,212.7,fba9d9fe-fa23-40c4-91c0-ee66e853ac07,RB,NY Giants,sit_and_go,NaN,Devontae Booker,12,16,4,2021-07-18 20:38:10,2021-07-18,2021,2021-07-18,214
15118,95e7fe68-0c37-4fbf-a706-73e5a081ae3f,2021-07-18 21:30:54,4c02707f-5f32-4b0b-9e50-f3f4aa8fca71,215,215.5,fba9d9fe-fa23-40c4-91c0-ee66e853ac07,RB,Los Angeles Rams,sit_and_go,NaN,Sony Michel,12,18,11,2021-07-18 20:38:10,2021-07-18,2021,2021-07-18,215


In [18]:
##############################################################################################
############################# Create Custom Lookups ##########################################
##############################################################################################

OUTPUT_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model\
/data/2021'
export_lookups(df_w_rank, OUTPUT_FOLDER, 'lookups_2021.csv')


In [11]:
##############################################################################################
############################### Create Custom Ranks ##########################################
##############################################################################################

def create_custom_ranks(df: pd.DataFrame) -> pd.DataFrame:
    """
    Uses the derived rank from a draft to create a custom rankings board
    for drafts where the actual ranks are not available.
    """

    # df = df_w_rank.copy()
    df = df_w_rank.copy()

    # Required to limit each day to just one set of rankings.
    # Since there will be players that don't get drafted in every
    # draft in a day, there will be days with more players than the size of a draft
    by_vars = ['ranks_draft_date', 'full_name', 'team_name', 'position']
    df['day_player_no'] = 1
    df['day_player_no'] = df.fillna('n/a').groupby(by_vars)['day_player_no'].cumsum()

    df = df.loc[df['day_player_no'] == 1]

    # Need to update the rank_derived field by ranks_draft_date
    df = drafts._add_rank_derived(df, by_var='ranks_draft_date')

    keep_vars = ['full_name', 'position', 'team_name', 'projection_adp'
                , 'rank_derived', 'ranks_draft_date']
    rename_vars = {'full_name': 'player', 'position': 'pos', 'team_name': 'team'
                , 'projection_adp': 'adp', 'rank_derived': 'rank', 'ranks_draft_date': 'date'}
    df = df[keep_vars].rename(columns=rename_vars)
    df['type'] = 'custom'

    return df

OUTPUT_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model\
/data/2021/player_ranks'

df = create_custom_ranks(df_w_rank)
df.to_csv(path.join(OUTPUT_FOLDER, 'df_player_ranks_2021.csv'), index=False)
df

,player,pos,team,adp,rank,date,type
11664,Christian McCaffrey,RB,Carolina Panthers,1.1,1,2021-05-12,custom
11665,Dalvin Cook,RB,Minnesota Vikings,2.1,2,2021-05-12,custom
11667,Saquon Barkley,RB,NY Giants,3.5,3,2021-05-12,custom
11666,Derrick Henry,RB,Tennessee Titans,3.9,4,2021-05-12,custom
11668,Alvin Kamara,RB,New Orleans Saints,5.4,5,2021-05-12,custom
...,...,...,...,...,...,...,...
638,Mark Ingram,RB,New Orleans Saints,212.0,212,2021-09-04,custom
646,Jared Goff,QB,Detroit Lions,212.1,213,2021-09-04,custom
641,Boston Scott,RB,Philadelphia Eagles,212.8,214,2021-09-04,custom
640,Jaret Patterson,RB,Washington Football Team,214.8,215,2021-09-04,custom


In [9]:
# Quick check that each date only uses one draft.
df_g = df.groupby('date').size().to_frame('num_players').reset_index()

df_g

,date,num_players
0,2021-05-12,216
1,2021-05-13,216
2,2021-05-15,216
3,2021-05-16,216
4,2021-05-21,216
5,2021-05-22,216
6,2021-05-23,216
7,2021-05-25,216
8,2021-05-27,216
9,2021-05-28,216
